##This is a minimal example for testing what the probability of a stimulus response PSTH having come from the distribution of baseline activity.

So you have a small number, <i>n</i>, of stimulus trial examples , and a large number of baseline examples. If you draw a large number (100,000) <i>n</i> examples from the baseline distribution, what percentage of those draws has a equal or higher firing rate than the stimulus mean for each bin?

In [ ]:
# initialize ipython "cluster" first.
# this connects client to the cluster workers.

from IPython.parallel import Client
C = Client(profile='default')
lbv = C.load_balanced_view()

In [1]:
# I'm defining the draw_and_mean as a nested function here because

def parfunc(rec_clu_tuple):
    from ephys_tools.analysis import laser_analysis as la
    import numpy as np
    
    def draw_and_mean(spike_array, n_bs, n_trials):
        # draw= np.empty_like(bl_sa)
        draw_indexes = np.random.randint(0, bl_sa.shape[1]-1, (n_bs, n_trials))
        psths = np.empty((n_bs, spike_array.shape[0]))
        for i in xrange(n_bs):
            draw_is = draw_indexes[i]
            draw = spike_array[:, draw_is]
            psths[i, :] = draw.mean(axis=1)
        return psths
    
    rec, clu = rec_clu_tuple # unpack rec and clu names.
    
    sa, _ = la.get_laser_train_psth_by_trial(rec, clu, 
                                             1, n_pulses=4, time_window_ms=100, pre_pad_ms=0, baseline_nsniffs=7)
    """
    sa is a tuple of two spike arrays: (stimulus_spike_array, baseline_spike_array)
    spike arrays are each numpy.array of shape (n_bins, n_trials) (for baseline it's actually n_sniffs instead of trials)
    spike arrays here are normalized (Hz).
    """
    
    stimulus_sa, baseline_sa = sa  #unpack tuple.
    
    response_mean = stimulus_sa.mean(axis=1)
    baseline_sa = sa[1]
    ntr = len(stimulus_sa) # number of trials in stimulus.
    baseline_boot = draw_and_mean(baseline_sa, 100000, ntr)

    ps = np.empty_like(response_mean)
    for j in xrange(len(response_mean)):
        r = response_mean[j]
        base = baseline_boot[:, j]
    #     print base.shape
        p = np.sum(base >= r) / len(base)
        ps[j] = p
    ps = np.asarray(ps)
    

    
    return ps

In [ ]:
"""all_rec_names is a list of """

rec_clu_tuples = [('rec1', 'clu1'),
                  ('rec1', 'clu2'),
                  ('recN', 'cluN')]

ps = lbv.map_sync(parfunc, rec_clu_tuples)


"""returns a list of probability arrays"""